In [1]:
import sys
sys.path.append('..')
from sentiment_predict.predict_sentiment_tools import *

In [2]:
posts = get_targets_posts("ETH")
result = get_sentiment_score_seq(posts)
result

[2.3218116766353547,
 2.0713743240309253,
 2.329922004157791,
 2.195468353491976,
 2.1328687270754787]

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import load_dataset

# Load dataset from Hugging Face
dataset = load_dataset("arad1367/Crypto_Fundamental_News")
df = pd.DataFrame(dataset['train'])

label_mapping = {"positive": 2, "neutral": 1, "negative": 0}
df["label"] = df["label"].map(label_mapping)

train_texts, temp_texts, train_labels, temp_labels = train_test_split(
    df["text"].tolist(), df["label"].tolist(), test_size=0.2, random_state=42
)

val_texts, test_texts, val_labels, test_labels = train_test_split(
    temp_texts, temp_labels, test_size=0.5, random_state=42
)

print(f"Training samples: {len(train_texts)}")
print(f"Validation samples: {len(val_texts)}")
print(f"Test samples: {len(test_texts)}")

2025-02-11 14:08:16 - numexpr.utils - INFO - Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2025-02-11 14:08:16 - numexpr.utils - INFO - NumExpr defaulting to 8 threads.
2025-02-11 14:08:17 - datasets - INFO - PyTorch version 2.5.1 available.
2025-02-11 14:08:17 - datasets - INFO - Polars version 0.20.8 available.
2025-02-11 14:08:17 - datasets - INFO - TensorFlow version 2.17.0 available.


Training samples: 10200
Validation samples: 1275
Test samples: 1275


In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import DataLoader

# Load tokenizer and model
model_name = "microsoft/deberta-v3-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name, 
    num_labels=3,
    ignore_mismatched_sizes=True
)

2025-02-11 14:08:39 - tensorflow - WARNING - From D:\Anaconda3\Lib\site-packages\tf_keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=512)

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

train_dataset = CustomDataset(train_encodings, train_labels)
val_dataset = CustomDataset(val_encodings, val_labels)
test_dataset = CustomDataset(test_encodings, test_labels)

In [6]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    save_total_limit=1,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

comet_ml version 3.43.0 is installed, but version 3.43.2 or higher is required. Please update comet_ml to the latest version to enable Comet logging with pip install 'comet-ml>=3.43.2'.


In [7]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,1.060900,1.055535
2,1.089400,1.041521
3,1.055500,1.042430


TrainOutput(global_step=3825, training_loss=1.0529288278218187, metrics={'train_runtime': 11510.4571, 'train_samples_per_second': 2.658, 'train_steps_per_second': 0.332, 'total_flos': 1651169085930000.0, 'train_loss': 1.0529288278218187, 'epoch': 3.0})

In [8]:
trainer.evaluate(test_dataset)

{'eval_loss': 1.0389422178268433,
 'eval_runtime': 6.9342,
 'eval_samples_per_second': 183.871,
 'eval_steps_per_second': 23.074,
 'epoch': 3.0}

In [9]:
def predict(text):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    inputs = {key: val.to(device) for key, val in inputs.items()}

    model.to(device)
    model.eval()

    with torch.no_grad():
        outputs = model(**inputs)

    probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
    predicted_index = torch.argmax(probabilities).item()
    confidence = probabilities[0, predicted_index].item()
    predicted_label = list(label_mapping.keys())[list(label_mapping.values()).index(predicted_index)]

    return predicted_label, confidence

In [10]:
post = get_targets_posts("btc")

In [11]:
texts = []
for item in post:
    content = item.__dict__['selftext']
    title = item.__dict__['title']
    texts.append((title, content))

In [12]:
predict(" ".join(texts[4]))

('positive', 0.464051753282547)